In [4]:
%pwd

'/Users/sabahatatta/Documents/GenAI_Projects/Medical-ChatBot-GenAI/research'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='/Users/sabahatatta/Documents/GenAI_Projects/Medical-ChatBot-GenAI/data')

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks


In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [12]:
#!pip install --upgrade sentence-transformers huggingface-hub
embeddings=download_hugging_face_embeddings()

/var/folders/sw/h5bx002d2j30zn4m0fl58g7m0000gn/T/ipykernel_5146/1196424635.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/opt/anaconda3/envs/medBot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [14]:
query_result

[-0.034477271139621735,
 0.031023167073726654,
 0.006734965834766626,
 0.0261090025305748,
 -0.03936200961470604,
 -0.1603025197982788,
 0.06692396849393845,
 -0.006441435776650906,
 -0.047450531274080276,
 0.014758886769413948,
 0.07087533921003342,
 0.05552756041288376,
 0.019193314015865326,
 -0.026251325383782387,
 -0.010109515860676765,
 -0.02694052830338478,
 0.02230745553970337,
 -0.022226618602871895,
 -0.14969263970851898,
 -0.01749310828745365,
 0.007676252163946629,
 0.05435231328010559,
 0.003254471579566598,
 0.031725961714982986,
 -0.08462142944335938,
 -0.029405971989035606,
 0.05159565061330795,
 0.0481240451335907,
 -0.003314791014418006,
 -0.05827918276190758,
 0.04196928068995476,
 0.022210730239748955,
 0.128188818693161,
 -0.022338921204209328,
 -0.011656280606985092,
 0.06292837113142014,
 -0.03287630155682564,
 -0.09122605621814728,
 -0.03117530420422554,
 0.05269954726099968,
 0.04703482985496521,
 -0.0842030793428421,
 -0.030056189745664597,
 -0.020744809880852

In [15]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [16]:
# Access the API key
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# Check if the API key is loaded (for debugging purposes)
if not PINECONE_API_KEY:
    raise ValueError("API key not found. Ensure it's set in the .env file.")

In [17]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medbot"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '2a33dbf0dfc9e3415c359cf9f52fd896', 'Date': 'Mon, 30 Dec 2024 08:32:43 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [18]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone.vectorstores import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [19]:
# Load Existing index 
from langchain_pinecone.vectorstores import PineconeVectorStore

# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [20]:
docsearch

In [21]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [22]:
retrieved_docs = retriever.invoke("What is Acne?")

In [23]:
retrieved_docs

[Document(id='a0958fad-bcb2-4052-8bbe-80f0606f6de8', metadata={'page': 39.0, 'source': '/Users/sabahatatta/Documents/GenAI_Projects/Medical-ChatBot-GenAI/data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='cbc3c782-1559-48c2-aa0a-65a03936df8e', metadata={'page': 39.0, 'source': '/Users/sabahatatta/Documents/GenAI_Projects/Medical-ChatBot-GenAI/data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='c4e22538-e4a9-490d-965c-070475cd546a', metadata={'page': 38.0, 'source': '/Users/sabahatatta/Documents/GenAI_Projects/Medical-ChatBot-GenAI/data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Research

In [24]:
# Access the OpenAI API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [25]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [26]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [27]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [29]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])



Acromegaly is a disorder caused by the overproduction of a chemical from the pituitary gland, resulting in excessive growth of bone and soft tissue. It can also cause other health issues throughout the body. It is also known as gigantism when it occurs during childhood or adolescence.


In [30]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])


I don't know.
